# [Helm](https://helm.sh/docs/)

Helm é o gerenciador de pacotes do Kubernetes.

Já vimos como criar cada recurso separadamente. O Helm realiza a gestão dos recursos (ns, deploy, svc, ...) desta forma, com apenas um comando, podemos implantar, alterar e excluir todo nosso projeto.

In [ ]:
helm version

Para criar o helm, executamos o comando abaixo:  

```
helm create \<nome-aplicacao>
```

Este comando irá criar a estrutura abaixo:
```
<nome-aplicacao>/
├── Chart.yaml      # Informações Técnicas
├── values.yaml     # Variáveis e Valores
└── templates/      # Modelos de Deployments, Services e todos outros recursos
```

[Helm do Projeto front-app](https://github.com/kdop-dev/front-app/tree/master/helm/front-app)

## Preparando o acesso ao Ambiente

In [ ]:
export KUBECONFIG=~/kubeconfig
export myNamespace=<seu namespace>
mkdir -p ~/work/inovacao
cd ~/work/inovacao
echo $PWD

## Projeto front-app

1. Publicando o Projeto front-app

    Para a publicar a aplicação via helm, podemos utilizar duas formas:
    * **helm install** - Comando para publicação da aplicação.
    * **helm upgrade** - Comando para atualização da aplicação. Passando o parâmetro **--install** ele publica a aplicação, caso não exista.

In [ ]:
#Informe um nome para o namespace
myNamespace=""

Vamos também modificar o arquivo [values.yaml](work/inovacao/front-app/helm/front-app/values.yaml) para ajustar o caminho da nossa URL.

> Você pode criar um outro arquivo de valores e sobreescrever os valores padrão do arquivo `values.yaml` da aplicação, basta copiá-lo e manter apenas as variáveis que serão modificadas. Para usá-lo passe a opção `-f my-values.yaml` no comando `helm install` ou `helm upgrade` Esta é a forma mais comun de customização dos helm charts.

In [ ]:
#helm install --namespace $myNamespace --create-namespace front-app front-app/helm/front-app
helm upgrade --install --namespace $myNamespace --create-namespace front-app front-app/helm/front-app

2. Listando os helms do namespace

In [ ]:
helm list --namespace $myNamespace 

3. Consultando os recursos do namespace

In [ ]:
kubectl get all -n $myNamespace

4. Verificando o problema no POD

In [ ]:
kubectl describe pod <insert-pod-name-here> -n $myNamespace 

* O POD necessita da aplicação back-app para poder ser executada.
  * **livenessProbe**: Verifica a saúde do container. Somente se o container estiver em saudável, o POD entrará em execução.    
```yaml
        livenessProbe:
            httpGet:
              path: /health
              port: http
            initialDelaySeconds: 3 # Tempo para realizar a primeira validação
            periodSeconds: 10      # Tempo em que será realizada a sondagem da atividade.
```
  </br>
  * **readinessProbe**: Indica se o contêiner está pronto para atender às solicitações. Se a análise de prontidão falhar, o controlador de terminais removerá o endereço IP do Pod dos terminais de todos os Serviços que correspondem ao Pod. 
```yaml
        readinessProbe:
            httpGet:
              path: /health
              port: http
            initialDelaySeconds: 3 # Tempo para realizar a primeira validação
            periodSeconds: 10      # Tempo em que será realizada a sondagem da atividade.
```
  </br>
  * **startupProbe**: Indica se o aplicativo no contêiner foi iniciado. Todas as outras análises serão desativadas se uma sonda de inicialização for fornecida, até que seja bem-sucedida. Se a sonda de inicialização falhar, o kubelet matará o Container e o Container será sujeita à sua política de reinicialização.
```yaml
        startupProbe:
            httpGet:
              path: /health
              port: http
            initialDelaySeconds: 3 # Tempo para realizar a primeira validação
            periodSeconds: 10      # Tempo em que será realizada a sondagem da atividade.
```
</br>
    Pode-se verificar a saúde do POD de 3 formas:  </br>
    1. executar um comando dentro de um contêiner   </br>    
    2. fazer uma solicitação HTTP em um contêiner  </br>
    (Código maior ou igual a 200 e menor que 400 indica sucesso. Qualquer outro código indica falha.) </br>
    3. abrir um soquete TCP em um contêiner.


### URLs da aplicação:

In [ ]:
echo http://learn.kdop.net/adsantos/front-app/index
echo https://learn.kdop.net/adsantos/front-app/index
echo http://kdop-learn.krthomolog.com.br/adsantos/front-app/index
echo https://kdop-learn.krthomolog.com.br/adsantos/front-app/index

### Como as requisições chegam aos containers?

Existem algumas alternativas para expor nossa aplicação utilizando [serviços](https://kubernetes.io/docs/concepts/services-networking/service/), tais como ClusterIP, LoadBalancer, NodePort, ExternalName. O Ingress não é um serviço é uma configuração para uma aplicação (ingress controller) direcionar o tráfego para o nosso serviço.

![](media/nginx-ingress.png)

Fonte para os ícones: <https://github.com/kubernetes/community/tree/master/icons>

No namespace ingress-nginx está instalado o Ingress Controller, ele é um [Servidor Web Nginx](https://github.com/nginxinc/kubernetes-ingress) modificado e integrado com o kubernetes. Existem outros Ingress Controller, para uma lista mais extensiva acesse [Kubernetes Ingress Controller Overview](https://medium.com/swlh/kubernetes-ingress-controller-overview-81abbaca19ec).

#### Ingress

Como configuramos o nosso ingress? Isso depende de como queremos que ele seja acessado, novamente existem várias opções, as mais comuns são:

* Domínio: front-app.com
* Subdominio: front-app.kdop.net
* Caminho: kdop.net/front-app

Para esta aplicação configuramos a terceira opção. Vamos abrir o arquivo [values.yaml](work/inovacao/front-app/helm/front-app/values.yaml) para entender como foi configurado.

```yaml
service:
  type: ClusterIP
  port: 5000

ingress:
  enabled: true
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /$2
  hosts:
    - host: learn.kdop.net
      paths:
      - /adsantos/front-app(/|$)(.*)
  tls: []
```

Esses parâmetros, quando aplicados pelo helm aos templates de serviço e ingress irão criar a seguinte configuração:

```yaml
---
# Source: front-app/templates/service.yaml
apiVersion: v1
kind: Service
metadata:
  name: front-app
  labels:
    helm.sh/chart: front-app-0.1.0
    app.kubernetes.io/name: front-app
    app.kubernetes.io/instance: front-app
    app.kubernetes.io/version: "1.16.0"
    app.kubernetes.io/managed-by: Helm
spec:
  type: ClusterIP
  ports:
    - port: 5000
      targetPort: http
      protocol: TCP
      name: http
  selector:
    app.kubernetes.io/name: front-app
    app.kubernetes.io/instance: front-app
---
# Source: front-app/templates/ingress.yaml
apiVersion: networking.k8s.io/v1beta1
kind: Ingress
metadata:
  name: front-app
  labels:
    helm.sh/chart: front-app-0.1.0
    app.kubernetes.io/name: front-app
    app.kubernetes.io/instance: front-app
    app.kubernetes.io/version: "1.16.0"
    app.kubernetes.io/managed-by: Helm
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /$2
spec:
  rules:
    - host: "learn.kdop.net"
      http:
        paths:
          - path: /adsantos/front-app(/|$)(.*)
            backend:
              serviceName: front-app
              servicePort: 5000
```

> Você pode imprimir o que será criado pelo helm com o comando `helm template` no lugar de `helm install`.

> Sobre TLS ver [README-ingresss-with-tls.md](README-ingresss-with-tls.md).

Um serviço do tipo **ClusterIP** foi criado e seu seletor são os PODs com labels `app.kubernetes.io/name` e `app.kubernetes.io/instance` com o valor `front-app`. Não importa quantas réplicas existirem, o serviço irá direcionar o tráfego, na forma de _round-robin_, para cada uma delas.

Mas _ClusterIP_ é um tipo de serviço que só é visível dentro do cluster, pode ser referênciado pelos containers através do seu nome e porta, neste caso `ping front-app` resultaria no retorno do IP associado ao serviço e `telnet front-app 5000` em uma conexão de sucesso.

Para acessar os containers de fora do cluster foi criado uma configuração tipo Ingress, nela foi definido a configuração do frontend que deverá direcionar o tráfego para o backend. Neste cenário, a URL que irá direcionar o tráfego para os nossos containers é http://learn.kdop.net/adsantos/front-app e qualquer caminhos e parâmetros adicionais serão repassados para os nossos containers na forma http://front-app:5000/<o resto da url>.

In [ ]:
kubectl describe ingress -n $myNamespace

Mas para essa configuração funcionar é necessário que um Ingress-Controller esteja instalado e lendo o seu namespace a procura desse tipo de configuração.
    
Neste cluster, o Ingress-Controller instalado é o Nginx, um servidor de web, que funciona como proxy, com um LoadBalancer externo direcionando o tráfego.

#### Ingress controller

In [ ]:
kubectl get all -n ingress-nginx

O serviço `ingress-nginx-controller` é do tipo **LoadBalancer** e seu endereço externo foi configurado em um DNS (AWS Route 53) para apontar para o seu IP.

In [ ]:
kubectl describe service/ingress-nginx-controller -n ingress-nginx

Aqui você pode obter o IP do Ingress-Controller e a quais portas ele esta associado, neste caso 80 (http) e 443 (https).

Vamos olhar o deployment do ingress controller para entender como o certificado foi associado.

In [ ]:
kubectl get deployment.apps/ingress-nginx-controller -n ingress-nginx -o yaml

E o secret contém o certificado (base 64).

In [ ]:
kubectl describe secret/krthomolog-secret -n ingress-nginx

Para mais informações: <https://kubernetes.github.io/ingress-nginx/user-guide/tls/>

[<img align="left" src="media/voltar.png" width="100"/>](02_Kubernetes.ipynb) [<img align="right" src="media/avancar.png" width="100"/>](04_Back.ipynb)
